In [1]:
import time
from utilities_s7 import *
from utilities_s8 import *
from plotting_module import *

## 7 Incorporating Conditional Information

### 7.1 Approach I

#### Basic problem:

\begin{equation}
\min_{M\geq0}{\mathbb E}\left[Mg(X)\right]
\end{equation}
*subject to constraints:*
\begin{align*}
&{\mathbb E}\left[M \log M\right] \leq \kappa, \\
&{\mathbb E}\left[B^j M Y\right] = 0, \\
&{\mathbb E}\left[M\right] = 1.
\end{align*}

#### Dual problem:

For computational purposes, we solve the dual problem after minimizing over $M$.  

\begin{equation*}
\sup_{\xi>0}\max_{\hat{\lambda}_j}    - \xi \log {\mathbb E} \left[ \exp\left( - {\frac 1 {\xi}} \left[ g(X) + \sum_{j}\hat{\lambda}_j \cdot YB^j \right] \right)\right]  -  \xi \kappa  
\end{equation*}

$\hat{\lambda}$ and $\xi$ are multipliers on the moment condition and relative entropy constraints. The implied solution for the distorted probability is:

\begin{equation*}
M^* = \frac{\exp\left[-\frac{1}{\xi^*}\left(g(X) + \hat{\lambda}^*\cdot f(X)\right)\right]}{\mathbb E \left( \exp\left[-\frac{1}{\xi^*}\left(g(X) + \hat{\lambda}^*\cdot f(X)\right)\right]\right)}
\end{equation*}


In [ ]:
# Set model paramters
tol = 1e-14
max_iter = 1000

solver_s7 = StaDivConstraint(tol,max_iter)
approach = 1

# Calculate lower bound for k
print("---Approach I---")
min_k_approach_1 = solver_s7.cal_min_k(approach)['result']
print("Lower Bound for k: %s" % min_k_approach_1)

# Set k
ratio = 1.1
k = min_k_approach_1 * ratio
print("We set k (%sx minimum) = %s" % (ratio,k))
result_approach_1 = solver_s7.solve(k,approach)
result_approach_1_upper = solver_s7.solve(k,approach,lower=False)

# Print probabilities
print("\n")
print("--- Distribution (Original) ---")
print(result_approach_1['π'])
print(" ")
print("--- Distribution (Distorted, for the lower bound problem) ---")
print(result_approach_1['π_tilde'])
print(" ")
print("--- Distribution (Distorted, for the upper bound problem) ---")
print(result_approach_1_upper['π_tilde'])

# Print conditional moments & bounds
print("\n")
print("--- Conditional Moment (Original) ---")
print("E[g(X)|state 1] = %s " % result_approach_1['moment_cond'][0])
print("E[g(X)|state 2] = %s " % result_approach_1['moment_cond'][1])
print("E[g(X)|state 3] = %s " % result_approach_1['moment_cond'][2])
print(" ")
print("--- Conditional Moment (Lower Bound) ---")
print("E[Mg(X)|state 1] = %s " % result_approach_1['moment_bound_cond'][0])
print("E[Mg(X)|state 2] = %s " % result_approach_1['moment_bound_cond'][1])
print("E[Mg(X)|state 3] = %s " % result_approach_1['moment_bound_cond'][2])
print(" ")
print("--- Conditional Moment (Upper Bound) ---")
print("E[Mg(X)|state 1] = %s " % result_approach_1_upper['moment_bound_cond'][0])
print("E[Mg(X)|state 2] = %s " % result_approach_1_upper['moment_bound_cond'][1])
print("E[Mg(X)|state 3] = %s " % result_approach_1_upper['moment_bound_cond'][2])

# Print unconditional moment & bounds
print("\n")
print("--- Unconditional Moment (Original) ---")
print("E[g(X)] = %s" % result_approach_1['moment'])
print(" ")
print("--- Unconditional Moment (Lower Bound) ---")
print("E[Mg(X)] = %s" % result_approach_1['moment_bound'])
print(" ")
print("--- Unconditional Moment (Upper Bound) ---")
print("E[Mg(X)] = %s" % result_approach_1_upper['moment_bound'])

# Print conditional relative entropy
print("\n")
print("--- Conditional Relative Entropy (for the lower bound problem) ---")
print("E[MlogM|state 1] = %s " % result_approach_1['RE_cond'][0])
print("E[MlogM|state 2] = %s " % result_approach_1['RE_cond'][1])
print("E[MlogM|state 3] = %s " % result_approach_1['RE_cond'][2])
print(" ")
print("--- Conditional Relative Entropy (for the upper bound problem) ---")
print("E[MlogM|state 1] = %s " % result_approach_1_upper['RE_cond'][0])
print("E[MlogM|state 2] = %s " % result_approach_1_upper['RE_cond'][1])
print("E[MlogM|state 3] = %s " % result_approach_1_upper['RE_cond'][2])

# Print unconditional relative entropy
print("\n")
print("--- Unconditional Relative Entropy (for the lower bound problem) ---")
print("E[MlogM] = %s" % result_approach_1['RE'])
print(" ")
print("--- Unconditional Relative Entropy (for the upper bound problem) ---")
print("E[MlogM] = %s" % result_approach_1_upper['RE'])

### 7.3 Approach III

#### Basic problem:

\begin{equation}
\min_{M\geq0}{\mathbb E}\left[Mg(X)\mid\mathfrak{F}_0\right]
\end{equation}
*subject to constraints:*
\begin{align*}
&{\mathbb E}\left[M \log M\mid\mathfrak{F}_0\right] \leq \kappa, \\
&{\mathbb E}\left[M  Y\mid\mathfrak{F}_0\right] = 0, \\
&{\mathbb E}\left[M\mid\mathfrak{F}_0\right] = 1.
\end{align*}

#### Dual problem:
For computational purposes, we solve the dual problem after minimizing over $M$.  

\begin{equation*}
\sup_{\xi>0}\max_{\hat{\lambda}}    - \xi \log {\mathbb E} \left[ \exp\left( - {\frac 1 {\xi}} \left[ g(X) + \hat{\lambda} \cdot Y \right] \right)\mid\mathfrak{F}_0\right]  -  \xi \kappa  
\end{equation*}

$\hat{\lambda}$ and $\xi$ are multipliers on the moment condition and relative entropy constraints. 

In [ ]:
# Set model paramters
tol = 1e-12
max_iter = 1000

solver_s7 = StaDivConstraint(tol,max_iter)
approach = 3
state = 1

# Calculate lower bound for k
print("---Approach III, condition on state %s---" % state)
min_k_approach_3 = solver_s7.cal_min_k(approach,state)['result']
print("Lower Bound for k: %s" % min_k_approach_1)

# Set k
k = min_k_approach_3 * 1.1
moment_bound_approach_3 = solver_s7.solve(k,approach,state)['result']
print("\n")
print("---Setting and Results---")
print("k = %s" % k)
print("E[Mg(X)|state %s]= %s" % (state,moment_bound_approach_3))

## 8 Intertemporal Divergence Constraints

### Proposition 8.6
Problem 8.4 can be solved by finding the solution to:

\begin{equation}
\epsilon = \min_\hat{\lambda}\mathbb E \left(\exp \left[-\frac{1}{\xi}g(X_1)+\hat{\lambda}\cdot f(X_1)\right]\left( \frac{e_1}{e_0}\right) \mid \mathfrak{F}_0\right)
\end{equation}

*where*
\begin{align*}
\mu &= -\xi \log \epsilon,\\
v_0 &= -\xi \log e_0.
\end{align*}

Denote $e^*$, $\hat{\lambda}^*$ as the solution to the above optimization problem. The implied solution for the probablity distortion is:

\begin{equation}
M_1^* = \frac{\exp \left[-\frac{1}{\xi}g(X_1)+\hat{\lambda}^*(Z_0)\cdot f(X_1)\right]e_1^*}{\epsilon^*e_0^*}
\end{equation}

In [ ]:
# Set model paramters

ξ = 0.2  # 10% higher than the minimum RE
tol = 1e-10

# ξ = 0.14 # 20% higher than the minimum RE
# tol = 1e-10

# ξ = 100.  # Minimal RE
# tol = 1e-12

solver_s8 = InterDivConstraint(tol,max_iter=1000)

time_start = time.time() 
result = solver_s8.iterate(ξ)
result_upper = solver_s8.iterate(ξ,lower=False)

# Print iteration information
print("--- Iteration Ends ---")
print("ξ = %s" % ξ)
print("Time spent: %s seconds ---" % (round(time.time()-time_start,4)))
# print("Numer of iterations: %s ---" % result['count'])

# Print converged parameter results
print("\n")
print("--- Converged vlues for the lower bound problem ---")
print("ϵ: %s" % result['ϵ'])
print("e: %s" % result['e'])
print("λ: %s" % result['λ'])
print("μ: %s" % result['μ'])

print(" ")
print("--- Converged vlues for the upper bound problem ---")
print("ϵ: %s" % result_upper['ϵ'])
print("e: %s" % result_upper['e'])
print("λ: %s" % result_upper['λ'])
print("μ: %s" % result_upper['μ'])

# Print E[M|state k]
print("\n")
print("--- Check 1 (lower bound problem) ---")
print("E[M|state 1] = %s " % result['E_M_cond'][0])
print("E[M|state 2] = %s " % result['E_M_cond'][1])
print("E[M|state 3] = %s " % result['E_M_cond'][2])

print(" ")
print("--- Check 1 (upper bound problem) ---")
print("E[M|state 1] = %s " % result_upper['E_M_cond'][0])
print("E[M|state 2] = %s " % result_upper['E_M_cond'][1])
print("E[M|state 3] = %s " % result_upper['E_M_cond'][2])

# Print two ways of calculating moment bound
print("\n")
print("--- Check 2 (lower bound problem)---")
print("E[Mg(X)] = %s (directly using M)" % result['moment_bound'])
print("E[Mg(X)] = %s (indirectly using μ and RE)" % result['moment_bound_check'])
print("Difference: %s" % (result['moment_bound']-result['moment_bound_check']))

print(" ")
print("--- Check 2 (upper bound problem)---")
print("E[Mg(X)] = %s (directly using M)" % result_upper['moment_bound'])
print("E[Mg(X)] = %s (indirectly using μ and RE)" % -result_upper['moment_bound_check'])
print("Difference: %s" % (result_upper['moment_bound']+result_upper['moment_bound_check']))

# Print transition probability matrix under the original empirical probability
print("\n")
print("--- Transition Probability Matrix (Original) ---")
print(result['P'])

# Print transition probability matrix under distorted probability, lower bound
print(" ")
print("--- Transition Probability Matrix (Distorted, lower bound problem) ---")
print(result['P_tilde'])

# Print transition probability matrix under distorted probability, upper bound
print(" ")
print("--- Transition Probability Matrix (Distorted, upper bound problem) ---")
print(result_upper['P_tilde'])


# Print stationary distribution under the original empirical probability
print("\n")
print("--- Stationary Distribution (Original) ---")
print(result['π'])

# Print stationary distribution under distorted probability, lower bound
print(" ")
print("--- Stationary Distribution (Distorted, lower bound problem) ---")
print(result['π_tilde'])

# Print stationary distribution under distorted probability, upper bound
print(" ")
print("--- Stationary Distribution (Distorted, upper bound problem) ---")
print(result_upper['π_tilde'])

# Print relative entropy
print("\n")
print("--- Relative Entropy (lower bound problem) ---")
print("E[MlogM|state 1] = %s " % result['RE_cond'][0])
print("E[MlogM|state 2] = %s " % result['RE_cond'][1])
print("E[MlogM|state 3] = %s " % result['RE_cond'][2])
print("E[MlogM]         = %s " % result['RE'])

# Print relative entropy
print(" ")
print("--- Relative Entropy (Upper bound problem) ---")
print("E[MlogM|state 1] = %s " % result_upper['RE_cond'][0])
print("E[MlogM|state 2] = %s " % result_upper['RE_cond'][1])
print("E[MlogM|state 3] = %s " % result_upper['RE_cond'][2])
print("E[MlogM]         = %s " % result_upper['RE'])

# Print conditional moment & bounds
print("\n")
print("--- Conditional Moment (Original) ---")
print("E[g(X)|state 1]  = %s " % result['moment_cond'][0])
print("E[g(X)|state 2]  = %s " % result['moment_cond'][1])
print("E[g(X)|state 3]  = %s " % result['moment_cond'][2])
print(" ")
print("--- Conditional Moment (Lower bound) ---")
print("E[Mg(X)|state 1] = %s " % result['moment_bound_cond'][0])
print("E[Mg(X)|state 2] = %s " % result['moment_bound_cond'][1])
print("E[Mg(X)|state 3] = %s " % result['moment_bound_cond'][2])
print(" ")
print("--- Conditional Moment (Upper bound) ---")
print("E[Mg(X)|state 1] = %s " % result_upper['moment_bound_cond'][0])
print("E[Mg(X)|state 2] = %s " % result_upper['moment_bound_cond'][1])
print("E[Mg(X)|state 3] = %s " % result_upper['moment_bound_cond'][2])

# Print unconditional moment & bounds
print("\n")
print("--- Unconditional Moment (Original) ---")
print("E[g(X)]  = %s " % result['moment'])
print(" ")
print("--- Unconditional Moment (Lower bound) ---")
print("E[Mg(X)] = %s " % result['moment_bound'])
print(" ")
print("--- Unconditional Moment (Upper bound) ---")
print("E[Mg(X)] = %s " % result_upper['moment_bound'])

In [ ]:
time_start = time.time() 
entropy_moment_bounds_s8()
print("Time spent: %s seconds ---" % (round(time.time()-time_start,4)))

### Three more cases

\begin{equation}
\epsilon = \min_\hat{\lambda}\mathbb E \left(\exp \left[-\frac{1}{\xi}g(X_1)+\hat{\lambda}\cdot f(X_1)\right]\left( \frac{e_1}{e_0}\right) \mid \mathfrak{F}_0\right)
\end{equation}

*where*
\begin{align*}
\mu &= -\xi \log \epsilon,\\
v_0 &= -\xi \log e_0.
\end{align*}

#### 1. Set $g(X_1)=0$

\begin{equation}
\epsilon = \min_\hat{\lambda}\mathbb E \left(\exp \left[\hat{\lambda}\cdot f(X_1)\right]\left( \frac{e_1}{e_0}\right) \mid \mathfrak{F}_0\right)
\end{equation}

In [ ]:
tol = 1e-10
solver_s8 = InterDivConstraint(tol,max_iter=1000)
result = solver_s8.iterate_check_1()
μ = - np.log(result['ϵ'])
temp = μ - np.mean(solver_s8.g)
print('μ = %s (set ξ = 1)' % μ)
print('E[g(X)] = %s' % np.mean(solver_s8.g))
print('μ - E[g(X)] = %s' % temp)

#### 2. Set $f(X_1)=0$, $\xi=1$

\begin{equation}
\epsilon = \mathbb E \left(\exp \left[-g(X_1)\right]\left( \frac{e_1}{e_0}\right) \mid \mathfrak{F}_0\right)
\end{equation}

In [ ]:
tol = 1e-10
solver_s8 = InterDivConstraint(tol,max_iter=1000)
result = solver_s8.iterate_check_2()
temp = result['μ'] - result['moment']
print('μ = %s' % result['μ'])
print('E[g(X)] = %s' % result['moment'])
print('μ - E[g(X)] = %s' % temp)

#### 3. Include $g(X)-r$ in the moment conditions  

\begin{equation}
\epsilon = \min_{\hat{\lambda}_1,\hat{\lambda}_2}\mathbb E \left(\exp \left[\hat{\lambda}_1\cdot f(X_1)+\hat{
\lambda}_2(g(X_1)-r)\right]\left( \frac{e_1}{e_0}\right) \mid \mathfrak{F}_0\right)
\end{equation}

In [3]:
tol = 1e-10
solver_s8 = InterDivConstraint(tol,max_iter=1000)
time_start = time.time() 
result = solver_s8.iterate_check_3(r=0.01)
print("Time spent: %s seconds ---" % (round(time.time()-time_start,4)))
μ = - np.log(result['ϵ'])
print('μ = %s' % μ)

Time spent: 0.5263 seconds ---
μ = 0.07570560039323186
